In [1]:
import jupyter_client
import re
import sys

In [2]:
manager = jupyter_client.KernelManager()
manager.start_kernel()
client = manager.client()
client.start_channels()

In [3]:
msg = client.get_iopub_msg()
while not (msg["msg_type"] == "status" and msg["content"]["execution_state"] == "idle"):
    msg = client.get_iopub_msg()

In [4]:
def execute_cell(code):
    client.execute(code)
    msg = client.get_iopub_msg()
    text = ""
    while not (msg["msg_type"] == "status" and msg["content"]["execution_state"] == "idle"):
        #print(msg)
        if msg["msg_type"] == "stream":
            text += msg["content"]["text"]
        elif msg["msg_type"] == "error":
            sys.stderr.write(msg["content"]["evalue"])
        msg = client.get_iopub_msg()
    return text

In [5]:
#execute_cell('import random\nprint random.randint(0, 1) == 1 ? "world" : "zach"')

In [6]:
regex = re.compile(r"^\s*\* ", re.M)

In [7]:
#print(regex.sub("", "        * import jupyter_client"))
#print(regex.sub("", "import jupyter_client"))

In [8]:
comment_first_char = False
is_in_comment = False
py_code = None
for c in iter(lambda: sys.stdin.read(1), ""):
    if is_in_comment:
        if py_code == "":
            if c == '*':
                sys.stdout.write("/**")
                py_code = None
            else:
                py_code += c
        elif py_code == None:
            sys.stdout.write(c)
            if comment_first_char:
                comment_first_char = False
                if c == '/':
                    is_in_comment = False
            else:
                if c == '*':
                    comment_first_char = True
        else:
            if comment_first_char:
                comment_first_char = False
                if c == '/':
                    is_in_comment = False
                    sys.stdout.write(execute_cell(regex.sub("", py_code)))
                    #sys.stdout.write(regex.sub("", py_code))
                    py_code = None
                else:
                    py_code += '*' + c
            else:
                if c == '*':
                    comment_first_char = True
                else:
                    py_code += c
    else:
        if comment_first_char:
            comment_first_char = False
            if c == '*':
                is_in_comment = True
                py_code = ""
            else:
                sys.stdout.write('/')
                sys.stdout.write(c)
        else:
            if c == '/':
                comment_first_char = True
            else:
                sys.stdout.write(c)